In [ ]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

### Set variables

`%env` are best for bash

In [ ]:
# Set workding directory
%env wd=/home/sam/analyses/20191203_pgen_v074.a4_genome_feature_counts
wd="/home/sam/analyses/20191203_pgen_v074.a4_genome_feature_counts"

# File download commands
%env downloads_list=Panopea-generosa-vv0.74.a4-merged-2019-11-26-15-30-34.gff Pgenerosa_v074.fa.fai


%env wget_command=--directory-prefix=${wd} --quiety --no-directories --no-check-certificate https://owl.fish.washington.edu/halfshell/genomic-databank/Panopea-generosa-vv0.74.a4-merged-2019-11-26-15-30-34.gff3
%env wget_fai=https://owl.fish.washington.edu/halfshell/genomic-databank/Pgenerosa_v074.fa.fai

# Input/output files
%env org_merged_gff=Panopea-generosa-vv0.74.a4-merged-2019-11-26-15-30-34.gff
%env new_merged_gff=Panopea-generosa-v1.0.a4.gff3
%env exon_gff=Panopea-generosa-v1.0.a4.exon.gff3
%env exon_bed=Panopea-generosa-v1.0.a4.exon.bed
%env gene_gff=Panopea-generosa-v1.0.a4.gene.gff3
%env gene_bed=Panopea-generosa-v1.0.a4.gene.bed
%env introns_bed=Panopea-generosa-v1.0.a4.introns.bed
%env intergenic_bed=Panopea-generosa-v1.0.a4.intergenic.bed

# Programs
%env intersectbed=/home/sam/programs/bedtools-2.28.0/bin/intersectBed

# Set list of column header names
bed_header = ['scaffold', 'start', 'end']

### Import Python modules

In [ ]:
%%bash
mkdir --parents ${wd}

In [ ]:
cd {wd}

### Download files